In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
from mne.time_frequency import tfr_morlet
import copy

# -------------------------------
# User inputs
data_path = "your_preprocessed_data.fif"
raw = mne.io.read_raw_fif(data_path, preload=True)
raw.filter(1., 40.)

events = mne.find_events(raw)
print("Unique events:", np.unique(events[:,2]))

# Triggers given:
# Rest start: 6
# Rest end: 7
# Beep Test start: 1
# Beep Test finish: 3
# Mario Test start: 2
# Mario Test finish:4
# Key press: also 3 (conflicts with beep finish)

# We must distinguish key presses (3) from beep finish (also 3).
# Heuristic: The last '3' after beep test start and before next big event (like rest start or another condition start)
# is beep finish. All other '3' events in between are key presses.
# Similarly for mario: finish is 4, no conflict.

# Let's process events to separate them:
processed_events = []
sfreq = raw.info['sfreq']

# Group events into blocks:
all_codes = events[:,2]
all_samples = events[:,0]

# Identify beep and mario blocks:
# A beep block starts at code=1 and ends at last code=3 before next big event or rest.
# A mario block starts at code=2 and ends at code=4.

i = 0
while i < len(events):
    trig = all_codes[i]
    sample = all_samples[i]
    if trig == 1:  # beep start
        # collect all events until beep finish
        start_idx = i
        j = i+1
        potential_key_presses = []
        beep_finish_sample = None
        while j < len(events):
            if all_codes[j] in [6,7,1,2,4]: # big event means block end
                # we must have found beep finish before hitting these, if not last '3' was finish
                break
            if all_codes[j] == 3:
                # candidate for either key press or final beep finish
                potential_key_presses.append(j)
            j += 1

        # last '3' in potential_key_presses is beep finish, others are key presses
        if len(potential_key_presses) > 0:
            beep_finish_idx = potential_key_presses[-1]
            beep_finish_sample = all_samples[beep_finish_idx]
            # key presses = all but last in potential_key_presses
            for k_idx in potential_key_presses[:-1]:
                processed_events.append([all_samples[k_idx],0,999]) # code 999 = key press
            # add beep start and beep finish
            processed_events.append([sample,0,1]) # beep start
            processed_events.append([beep_finish_sample,0,3]) # beep finish
            i = beep_finish_idx + 1
        else:
            # no '3' found after beep start = incomplete?
            # just add beep start and move on
            processed_events.append([sample,0,1])
            i += 1

    elif trig == 2: # mario start
        processed_events.append([sample,0,2])
        # look for mario finish=4 and key presses=3?
        j = i+1
        while j < len(events) and all_codes[j] != 4:
            if all_codes[j] == 3:
                # here 3 unambiguously a key press or also beep finish in beep test?
                # We are in Mario test block, beep test ended earlier, so '3' here is key press
                processed_events.append([all_samples[j], 0, 999])
            j += 1
        # now j points to 4 (mario finish)
        if j < len(events) and all_codes[j] == 4:
            processed_events.append([all_samples[j], 0,4])
            i = j+1
        else:
            # no mario finish found?
            i = j

    elif trig == 6 or trig == 7:
        # rest start/end
        processed_events.append([sample,0,trig])
        i += 1
    else:
        # triggers outside known structure, just add them
        processed_events.append([sample,0,trig])
        i += 1

processed_events = np.array(processed_events, int)
print("Processed event codes:", np.unique(processed_events[:,2]))

# Now we have:
# 1 = Beep start
# 3 = Beep finish
# 999 = key press
# 2 = Mario start
# 4 = Mario finish
# 6 = rest start, 7 = rest end

raw.set_annotations(None) # if any
# If you want to do ERP analysis around key presses:
event_id = {
    'key_press':999
}
tmin, tmax = -0.2, 0.2
epochs_key = mne.Epochs(raw, processed_events, event_id=event_id,
                        tmin=tmin, tmax=tmax, baseline=(None,0),
                        preload=True)
evoked_key = epochs_key.average()
evoked_key.plot_joint(title='Key Press ERP')

# If you want to separate conditions:
# For beep test: between start(1) and finish(3), we have key presses
# For mario test: between start(2) and finish(4), we have key presses
# Extract key presses happening in beep blocks vs mario blocks
beep_blocks = []
mario_blocks = []

# Identify beep blocks and key press within them
for i, ev in enumerate(processed_events):
    if ev[2] == 1: # beep start
        start_sample = ev[0]
        # find finish
        finish_idx = np.where((processed_events[:,0]>start_sample) & (processed_events[:,2]==3))[0]
        if len(finish_idx)>0:
            finish_sample = processed_events[finish_idx[0],0]
            # key presses with 999 inside this interval
            presses = processed_events[(processed_events[:,0]>start_sample)&(processed_events[:,0]<finish_sample)&(processed_events[:,2]==999)]
            beep_blocks.append((start_sample, finish_sample, presses))
    elif ev[2]==2: # mario start
        start_sample = ev[0]
        finish_idx = np.where((processed_events[:,0]>start_sample) & (processed_events[:,2]==4))[0]
        if len(finish_idx)>0:
            finish_sample = processed_events[finish_idx[0],0]
            presses = processed_events[(processed_events[:,0]>start_sample)&(processed_events[:,0]<finish_sample)&(processed_events[:,2]==999)]
            mario_blocks.append((start_sample, finish_sample, presses))

# Now you can create epochs specifically for beep key presses and mario key presses if needed
beep_key_samples = np.concatenate([b[2][:,0] for b in beep_blocks if len(b[2])>0]) if beep_blocks else np.array([])
mario_key_samples = np.concatenate([b[2][:,0] for b in mario_blocks if len(b[2])>0]) if mario_blocks else np.array([])

def create_epochs_for_list(raw, samples, code, tmin=-0.2, tmax=0.2):
    if len(samples)==0:
        return None
    new_events = np.column_stack([samples, np.zeros(len(samples),int), np.full(len(samples), code,int)])
    return mne.Epochs(raw, new_events, event_id={f'condition_{code}':code},
                      tmin=tmin, tmax=tmax, baseline=(None,0),
                      preload=True)

epochs_beep_keys = create_epochs_for_list(raw, beep_key_samples, 101) # code 101 for beep keys
epochs_mario_keys = create_epochs_for_list(raw, mario_key_samples, 102) # code 102 for mario keys

if epochs_beep_keys is not None and epochs_mario_keys is not None:
    evoked_beep_keys = epochs_beep_keys.average()
    evoked_mario_keys = epochs_mario_keys.average()
    mne.viz.plot_compare_evokeds({'Beep Keys': evoked_beep_keys, 'Mario Keys': evoked_mario_keys},
                                 picks='Cz', title='Comparison of Key Press ERPs')

# Time-Frequency Analysis on key press epochs (e.g., beep_keys)
if epochs_beep_keys is not None:
    picks = mne.pick_channels(raw.info['ch_names'], include=['C3','Cz','C4'])
    freqs = np.arange(8,31,2)
    n_cycles = freqs/2.0
    power_beep = tfr_morlet(epochs_beep_keys, picks=picks, freqs=freqs,
                            n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3)
    power_beep.plot_topo(baseline=(None,0), mode='logratio', title='Beep condition TFR (Key Press)')

if epochs_mario_keys is not None:
    power_mario = tfr_morlet(epochs_mario_keys, picks=picks, freqs=freqs,
                             n_cycles=n_cycles, use_fft=True, return_itc=False, decim=3)
    power_mario.plot_topo(baseline=(None,0), mode='logratio', title='Mario condition TFR (Key Press)')

# Compare power or ERP components to literature:
# Check timing of negativity before key press (Readiness Potential) or post-movement beta rebound.
# Compare beep vs. mario to see if musical cue leads to stronger phase-locking or different ERP amplitude.

plt.show()
